# Practice PS06: Recommendations engines (interactions-based)

For this assignment we will build and apply an item-based and model-based collaborative filtering recommenders for movies. 

Author: <font color="blue">Josip Hanak</font>

E-mail: <font color="blue">josip.hanak@fer.hr</font>

Date: <font color="blue">10.11.2022</font>

# 1. The Movies dataset

We will use the same dataset as in ps05, the 25M version of [MovieLens DataSet](https://grouplens.org/datasets/movielens/) released in late 2019. We will use a sub-set containing only movies released in the 2000s, and only 10% of the users and all of their ratings.

* **MOVIES** are described in `movies-2000s.csv` in the following format: `movieId,title,genres`.
* **RATINGS** are contained in `ratings-2000s.csv` in the following format: `userId,movieId,rating`
* **TAGS** are contained in `tags.csv` in the following format: `userId,movieId,tag,timestamp`

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

# 1.1. Load the input files

In [1]:
# Leave this code as-is

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from math import*
from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import linear_kernel

In [2]:
# Leave this code as-is

FILENAME_MOVIES = "movies-2000s.csv"
FILENAME_RATINGS = "ratings-2000s.csv"
FILENAME_TAGS = "tags-2000s.csv"

In [3]:
# Leave this code as-is

movies = pd.read_csv(FILENAME_MOVIES, 
                    sep=',', 
                    engine='python', 
                    encoding='latin-1',
                    names=['movie_id', 'title', 'genres'])
display(movies.head(5))

ratings_raw = pd.read_csv(FILENAME_RATINGS, 
                    sep=',', 
                    encoding='latin-1',
                    engine='python',
                    names=['user_id', 'movie_id', 'rating'])
display(ratings_raw.head(5))

,movie_id,title,genres
0,2769,"Yards, The (2000)",Crime|Drama
1,3177,Next Friday (2000),Comedy
2,3190,Supernova (2000),Adventure|Sci-Fi|Thriller
3,3225,Down to You (2000),Comedy|Romance
4,3228,Wirey Spindell (2000),Comedy


,user_id,movie_id,rating
0,4,1,3.0
1,4,260,3.5
2,4,296,4.0
3,4,541,4.5
4,4,589,4.0


# 1.2. Merge the data into a single dataframe

Join the data into a single dataframe that should contain columns: user_id, movie_id, rating, timestamp, title, genders.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code from the previous practice that joined these three dataframes using "merge" into a single dataframe named "ratings". Print the first 5 rows of the resulting dataframe, which should contain columns "user_id", "movie_id", "rating", "title", and "genres".</font>

In [4]:
ratings = pd.merge(ratings_raw, movies,  how="inner", on="movie_id")
display(ratings.head(5))

,user_id,movie_id,rating,title,genres
0,4,3624,2.5,Shanghai Noon (2000),Action|Adventure|Comedy|Western
1,152,3624,3.0,Shanghai Noon (2000),Action|Adventure|Comedy|Western
2,171,3624,3.5,Shanghai Noon (2000),Action|Adventure|Comedy|Western
3,276,3624,4.0,Shanghai Noon (2000),Action|Adventure|Comedy|Western
4,494,3624,3.5,Shanghai Noon (2000),Action|Adventure|Comedy|Western


<font size="+1" color="red">Replace this cell with your code from the previous practice for "find_movies" that list movies containing a keyword</font>

In [5]:
def find_movies(keyword, dataframe):
    #return_movies = dataframe[dataframe['title'].str.contains(keyword)]
    #final = return_movies.iloc[0].movie_id
    for row in range(len(dataframe)):
        if keyword in dataframe.iloc[row].title:
            return dataframe.iloc[row].movie_id
    return 0

In [6]:
# LEAVE AS-IS

# For testing, this should print '59784'
find_movies("Kung Fu Panda (2008)", movies)

59784

The following function, which you can leave as-is, prints the title of a movie given its movie_id.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [7]:
# LEAVE AS-IS

def get_title(movie_id, movies):
    return movies[movies['movie_id'] == movie_id].title.iloc[0]

In [8]:
# LEAVE AS-IS

# For testing, should print "Kung Fu Panda (2008)")
print(get_title(59784, movies))

Kung Fu Panda (2008)


## 1.3. Count unique registers

Count the number of unique users and unique movies in the `ratings` variable. Use [unique()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.unique.html). Print also the total number of movies in the `movies` variable. Your code should print:

```
Number of users who have rated a movie : 12676
Number of movies that have been rated  : 2049
Total number of movies                 : 33168
```

Note that ratings are heavily concentrated on a few popular movies.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your own code to indicate the number of unique users and unique movies in the "ratings" variable.</font>

In [9]:
print("Number of users who have rated a movie : ",len(pd.unique(ratings.user_id)))
print("Number of users who have rated a movie : ",len(pd.unique(ratings.movie_id)))
print("Total number of movies                 : ", len(movies.movie_id))

Number of users who have rated a movie :  12676
Number of users who have rated a movie :  2049
Total number of movies                 :  33168


# 2. Item-based Collaborative Filtering

The two main types of interactions-based recommender system, also known as *collaborative filtering* algorithms are:

1. **User-based Collaborative Filtering**: To recommend items for user A, we first look at other users B1, B2, ..., Bk with a similar behavior to A, and aggregate their preferences. For instance, if all Bi like a movie that A has not watched, it would be a good candidate to be recommended. 


2. **Item-based Collaborative Filtering**: To recommend items for user A, we first look at all the items I1, I2, ..., Ik that the user A has consumed, and find items that elicit similar ratings from other users. For instnce, an item that is rated positively by the same users that rate positively the Ii items, and negatively by the same users that rate negatively the Ii items, would be a good candidate to be recommended.

In both cases, a similarity matrix needs to be built. For user-based, the **user-similarity matrix** will consist of some **distance metrics** that measure the similarity between any two pairs of users. For item-based, the **matrix** will measure the similarity between any two pairs of items.

As we already know, there are several metrics strategy for measure the "similarity" of two items. Some of the most used metrics are Jaccard, Cosine and Pearson. Meanwhile, Jaccard similarity is based on the number of users which have rated item A and B divided by the number of users who have rated either A or B (very useful for those use cases where there is not a numeric rating but just a boolean value like a product being bought), in Pearson and Cosine similarities we measure the similarity between two vectors.

For the purpose of this assignment, we will use **Pearson Similarity** and we will implement a **Item-based Collaborative filtering**.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

## 2.1. Data pre-processing

Firstly, create a new dataframe called "rated_movies" that is simply the "ratings" dataset with column genres removed using the [Drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) function.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to generate "rated_movies" and print the first five rows. This should have columns user_id, movie_id, rating, title</font>

In [7]:
rated_movies = ratings.drop(columns=["genres"])
display(rated_movies.head(5))

,user_id,movie_id,rating,title
0,4,3624,2.5,Shanghai Noon (2000)
1,152,3624,3.0,Shanghai Noon (2000)
2,171,3624,3.5,Shanghai Noon (2000)
3,276,3624,4.0,Shanghai Noon (2000)
4,494,3624,3.5,Shanghai Noon (2000)


Now, using the `rated_movies` dataframe, create a new dataframe named `ratings_summary` containing the following columns:

* movie_id
* title
* ratings_mean (average rating)
* ratings_count (number of people who have rated this movie)

You can use the following operations:

* Initialize `ratings_summary` to be only the movie_id and title of all movies in `rated_movies`
   * To group dataframe `df` by column `a` and keep only one unique row per value of `a`, use: `df.groupby('a').first()`
* Compute two series: `ratings_mean` and `ratings_count`:
   * To obtain a series with the average of column `a` for each distinct value of column `b` in dataframe `df`, use: `df.groupby(b)['a'].mean()`
   * To obtain a series with the count of column `a` for each distinct value of column `b` in dataframe `df`, use: `df.groupby(b)['a'].count()`
* Add these series to the `ratings_summary`
   * To add a series `s` with column name `a` to dataframe `df`, use: `df['a'] = s`
    
<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to generate "ratings_summary" and print the first 5 rows.</font>

In [8]:
ratings_summary = pd.DataFrame()
#ratings_summary = rated_movies[['movie_id', 'title']].copy()
ratings_summary = rated_movies.drop(columns=["user_id", "rating"],axis=1).groupby("movie_id").first()
ratings_summary["ratings_mean"] = rated_movies.groupby(rated_movies.movie_id)["rating"].mean()
ratings_summary["ratings_count"] = rated_movies.groupby(rated_movies.movie_id)["rating"].count()
display(ratings_summary.head(10))

,title,ratings_mean,ratings_count
movie_id,,,
2769,"Yards, The (2000)",3.122549,102
3177,Next Friday (2000),2.824000,125
3190,Supernova (2000),2.395683,139
3225,Down to You (2000),2.577273,110
3228,Wirey Spindell (2000),2.500000,2
3239,Isn't She Great? (2000),1.947368,19
3273,Scream 3 (2000),2.444664,759
3275,"Boondock Saints, The (2000)",3.870682,1071
3276,Gun Shy (2000),3.338710,31


To select from dataframe A those having column C larger or equal to N, you can do `A[A.C >= N]`.

To sort dataframe A by decreasing values of column C, you can do `A.sort_values(by='C', ascending=False)`.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with code to print the top 10 highest rated movies, considering only movies receiving at least 2000 ratings.</font>

In [12]:
display(ratings_summary[ratings_summary.ratings_count >= 2000].sort_values(by='ratings_mean', ascending=False).head(10))

,title,ratings_mean,ratings_count
movie_id,,,
5618,Spirited Away (Sen to Chihiro no kamikakushi) ...,4.215216,2458
6016,City of God (Cidade de Deus) (2002),4.186592,2133
4226,Memento (2000),4.158512,4476
4973,"Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le)...",4.097234,3687
4993,"Lord of the Rings: The Fellowship of the Ring,...",4.092530,5944
7153,"Lord of the Rings: The Return of the King, The...",4.083960,5449
5952,"Lord of the Rings: The Two Towers, The (2002)",4.083869,5449
7361,Eternal Sunshine of the Spotless Mind (2004),4.074589,3593
4011,Snatch (2000),4.034259,2700


<font size="+1" color="red">Repeat this, but this time consider movies receiving at least 3 ratings. What is the difference? How would you explain this?</font>

In [13]:
display(ratings_summary[ratings_summary.ratings_count >= 3].sort_values(by='ratings_mean', ascending=False).head(10))

,title,ratings_mean,ratings_count
movie_id,,,
5082,"Rumor of Angels, A (2000)",4.666667,6
27764,2LDK (2003),4.500000,3
31954,Beautiful City (Shah-re ziba) (2004),4.400000,5
5224,Promises (2001),4.388889,18
6775,Life and Debt (2001),4.333333,3
31856,Surplus: Terrorized Into Being Consumers (2003),4.333333,3
6672,War Photographer (2001),4.229167,24
5618,Spirited Away (Sen to Chihiro no kamikakushi) ...,4.215216,2458
7563,"Discovery of Heaven, The (2001)",4.200000,5


In [16]:
Movies who haven't got a decent amount of ratings are not objective, perhaps users who support the director
or ar just a fan of the type of the movie rated it and the general audience hadn't. That is why
they have an unusually good rating.

SyntaxError: unterminated string literal (detected at line 1) (1046736147.py, line 1)

## 2.2. Compute the user-movie matrix

Before calculating the **similarity matrix**, we create a table where columns are movies and rows are users, and each movie-user cell contains the score of that user for that movie.

We will use the [pivot_table](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html) function of Pandas, which receives a dataframe plus the variable that will make the rows, the variable that will make the columns, and the variable that will make the cells, and transform it into a matrix of the specified rows, columns, and cells.

For instance, if you have a dataframe D containing:

```
U V W
1 a 3.0
1 b 2.0
2 a 1.0
2 c 4.0
```

Calling `D.pivot_table(index='U', columns='V', values='W')` will create the following:

```
V  a   b   c
U
1 3.0 2.0 NaN
2 1.0 NaN 4.0
```

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with code to generate a "user_movie" matrix by calling "pivot_table" on "rated_movies". Print the first 10 rows. It might take about one minute to compute, depending on your computer.</font>

In [9]:
user_movie = rated_movies.pivot_table(index='user_id', columns='movie_id', values='rating')
display(user_movie.head(10))

movie_id,2769,3177,3190,3225,3228,3239,3273,3275,3276,3279,...,33138,33145,33148,33150,33152,33154,33158,33162,33164,33166
user_id,,,,,,,,,,,,,,,,,,,,,
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5
63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<font size="+1" color="red">Replace this a brief commentary indicating why do you think the "user_movie" matrix has so many "NaN" values. How do we call this characteristic of user ratings in recommender systems?</font>

In [15]:
I presume that most of the values are NaNs because each column has a large number of rows symbolizing every user.
It is illogical to think that every user rated every movie (for every movie a small subset of the total users rated it) and that
is why most of the fields are empty.

SyntaxError: invalid syntax (1375351603.py, line 1)

# 2.3. Explore some correlations in the user-movie matrix

Now let us explore whether correlations in this matrix make sense.

1. Locate the movie_id for the following three movies:
  * [Monsters, Inc. (2001)](https://en.wikipedia.org/wiki/Monsters,_Inc.) -- name this id_pivot
  * [Finding Nemo (2003)](https://en.wikipedia.org/wiki/Finding_Nemo) -- name this id_m1
  * [Talk to Her (Hable con Ella) (2002)](https://en.wikipedia.org/wiki/Talk_to_Her) -- name this id_m2
2. Obtain the ratings for each of these movies: `user_movie[movie_id].dropna()`. You will obtain a column, containing a series of ratings for each movie.
3. Consolidate these four series into a single dataframe: `ratings3 = pd.concat([s1, s2, s3], axis=1)`
4. Drop from `ratings3` all rows containing a *NaN*. This will keep only the users that have rated all the 3 movies.
5. Display the first 10 rows from this table.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with code to compute and display the first 10 rows of the "ratings3" table as described above.</font>

In [16]:
print(find_movies("Monsters, Inc", rated_movies))
print(find_movies("Finding Nemo", rated_movies))
print(find_movies("Hable con Ella", movies))


4886
6377
5878


In [10]:
id_pivot = user_movie[4886].dropna()
id_m1 = user_movie[6377].dropna()
id_m2 = user_movie[5878].dropna()
ratings3 = pd.concat([id_pivot, id_m1, id_m2], axis=1).dropna()

In [19]:
display(ratings3.head(10))

,4886,6377,5878
user_id,,,
859,4.5,4.0,5.0
1229,4.0,4.0,4.5
1281,2.5,2.5,3.0
1722,4.5,4.5,4.0
2004,4.0,3.0,3.5
2360,5.0,4.0,5.0
3726,5.0,5.0,2.5
3798,4.5,4.5,5.0
4590,4.5,4.0,2.0


To compute Pearson correlation, we use the [corr](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.corr.html) method.

To compute the correlation between two columns `a`, `b` in dataframe `df`, we use: `df[a].corr(df[b])`.

Compute the correlations between all pairs of columns of the `ratings3` table. You should display:

```
Similarity between 'Monsters, Inc. (2001)' and 'Finding Nemo (2003)': 0.56
Similarity between 'Monsters, Inc. (2001)' and 'Talk to Her (Hable con Ella) (2002)': 0.17
Similarity between 'Finding Nemo (2003)' and 'Talk to Her (Hable con Ella) (2002)': 0.24
```

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with code to compute all correlations between these three movies, as described above.</font>

In [20]:
print("Similarity between 'Monsters, Inc. (2001)' and 'Finding Nemo (2003)': ", round(ratings3[4886].corr(ratings3[6377]), 2))
print("Similarity between 'Monsters, Inc. (2001)' and 'Talk to Her (Hable con Ella) (2002): ", round(ratings3[4886].corr(ratings3[5878]),2))
print("Similarity between 'Finding Nemo (2003)' and 'Talk to Her (Hable con Ella) (2002)': ", round(ratings3[6377].corr(ratings3[5878]),2))

Similarity between 'Monsters, Inc. (2001)' and 'Finding Nemo (2003)':  0.56
Similarity between 'Monsters, Inc. (2001)' and 'Talk to Her (Hable con Ella) (2002):  0.17
Similarity between 'Finding Nemo (2003)' and 'Talk to Her (Hable con Ella) (2002)':  0.24


<font size="+1" color="red">Replace this cell with a brief commentary on the correlations you find.</font>

In [22]:
The correlations between Monsterc, Inc. and Finding Nemo is reasonably high which is logical because both are animated
movies with the same type of general genre. However with Talk To Her (Hable con Ella) the correlation is small which
is again logical because they have nothing in common (it is a drama and he previous two are animated adventure comedies).
The users who were rating like more animated movies than dramas.

SyntaxError: invalid syntax (1583887561.py, line 1)

Now let us take the first movie selected above, the one with movie_id `id_pivot`.

Select the column corresponding to this movie in `user_movies` and compute its correlation with all other columns in `user_movies`. This can be done with  [corrwith](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corrwith.html).

*Note 1*: You might receive a runtime warning on degrees of freedom and/or division by zero, which you can safely ignore. It simply means that there are some columns that have no elements in common with the given column, or only one element in common, and thus for which the correlation cannot be computed.

*Note 2*: Note that the similarities that you computed above with `corr` are limited to the set of users who rated all the three movies. Instead, the similarities that you compute below with `corrwith` include all users who rated the pivot plus at least one other movie. Hence, the results could be different.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with code to create a "similar_to_pivot" series that contains the computed correlations, droping the NaNs in the series.</font>

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [44]:
#pivot = user_movie[4886].dropna().to_frame()
pivot = id_pivot.to_frame()
similar_to_pivot = user_movie.corrwith(user_movie[4886]).dropna()
display(similar_to_pivot.head(10))
similar_to_pivot = similar_to_pivot.to_frame()

2769   -0.082710
3177    0.210484
3190    0.039882
3225    0.162232
3239   -0.438086
3273    0.236845
3275    0.122747
3276    0.694492
3279    1.000000
3285    0.006660
dtype: float64

In [45]:

similar_to_pivot["movie_id"] = similar_to_pivot.index
similar_to_pivot.rename(columns={similar_to_pivot.columns[0]:'corr'}, inplace=True)
#similar_to_pivot["corr"] = similar_to_pivot["0"]
display(similar_to_pivot.head(10))

,corr,movie_id
2769,-0.082710,2769
3177,0.210484,3177
3190,0.039882,3190
3225,0.162232,3225
3239,-0.438086,3239
3273,0.236845,3273
3275,0.122747,3275
3276,0.694492,3276
3279,1.000000,3279
3285,0.006660,3285


Next, create a dataframe `corr_with_pivot` by using `similar_to_pivot` and `ratings_summary`. This dataframe should have the following columns:

* corr - the correlation between each movie and the selected movie
* title
* ratings_mean
* ratings_count

To create a dataframe `df` from a series `s`, use: `df = pd.DataFrame(s, columns=['colname'])`. 

Keep only rows in which *ratings_count* > 500, i.e., popular movies. To filter a dataframe `df` and keep only rows having column `c` larger than `x`, use `df[df[c] > x]`.

Display the top 10 rows with the largest correlation. To select the largest `n` rows from dataframe `df` according to column `c`, use `df.sort_values(c, ascending=False).head(n)`. 

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with code to create a "corr_with_pivot" dataframe as specified above, and to print the 10 movies (rated 500 times or more) with the highest correlation with the selected movie.</font>

In [50]:
#similar_to_pivot = similar_to_pivot.to_frame()
corr_with_pivot = pd.merge(ratings_summary, similar_to_pivot,  how="inner", on="movie_id")
corr_with_pivot = corr_with_pivot[corr_with_pivot.ratings_count>500]
corr_with_pivot = corr_with_pivot.drop(columns=["movie_id"],axis=1)

In [51]:
display(corr_with_pivot.sort_values("corr", ascending=False).head(10))


,title,ratings_mean,ratings_count,corr
437,"Monsters, Inc. (2001)",3.850066,3775,1.000000
941,Finding Nemo (2003),3.862284,3765,0.639308
1446,"Incredibles, The (2004)",3.867771,3320,0.546784
291,Shrek (2001),3.768787,4591,0.471673
535,Ice Age (2002),3.546400,1972,0.465032
615,Lilo & Stitch (2002),3.576974,760,0.424311
77,Chicken Run (2000),3.476384,2096,0.423167
1304,Shrek 2 (2004),3.520989,2144,0.404075
184,"Emperor's New Groove, The (2000)",3.573889,900,0.402423
68,Shanghai Noon (2000),3.297443,1017,0.373850


<font size="+1" color="red">Replace this cell with a brief commentary about the movies you see on this list. What happens if you set the condition on *ratings_count* to a much larger value? What happens if you set it to a much smaller value?</font>

In [ ]:
The movies correlate very well to the selected movie. If we put ratings count as a much higher value it will probably 
be even more correct but fewer movies will be preseneted. If we put it as a smaller value it will probably be less correct

# 2.4. Implement the item-based recommendations

Now that we believe that this type of correlation sort of makes sense, let us implement the item-based recommender. We need all correlations between columns in `user_movie`.

To compute all correlations between columns in a dataframe, use [corr](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html). This function receives a matrix with *r* rows and *c* columns, and returns a square matrix of *c x c* containing all pair-wise correlations between columns.

**This process may take a few minutes.** Print the first 5 rows of the resulting matrix when done.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to compute all correlations between columns (movies) in the matrix user_movie. Store this in "item_similarity", and print the first 5 rows.</font>

In [52]:
item_similarity  = user_movie.corr()



In [53]:
display(item_similarity.head(5))

movie_id,2769,3177,3190,3225,3228,3239,3273,3275,3276,3279,...,33138,33145,33148,33150,33152,33154,33158,33162,33164,33166
movie_id,,,,,,,,,,,,,,,,,,,,,
2769,1.000000,0.115068,0.033721,-0.232268,NaN,-0.500000,0.197011,0.199514,0.250873,NaN,...,0.379980,0.878310,NaN,NaN,NaN,0.248126,0.180609,-0.085570,-0.408248,0.105671
3177,0.115068,1.000000,0.303820,0.559533,NaN,NaN,0.331191,0.167918,1.000000,NaN,...,0.546119,0.735767,-1.0,NaN,NaN,-0.221382,0.317475,0.014735,0.661989,0.185654
3190,0.033721,0.303820,1.000000,0.636361,NaN,-0.014315,0.146042,0.394293,-0.290397,NaN,...,0.246183,0.632026,NaN,NaN,NaN,0.378181,0.170926,0.022444,-0.073360,-0.054114
3225,-0.232268,0.559533,0.636361,1.000000,NaN,0.578414,0.347716,0.263671,-0.250313,NaN,...,-0.300376,0.318377,NaN,NaN,NaN,0.480173,0.750306,0.536828,0.753141,0.098748
3228,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Similarities between movies that do not have many ratings in common are unreliable. Fortunately, the `corr` method includes a parameter `min_periods` that establishes a minimum number of elements in common that two columns must have to compute the correlation.

Re-generate item_similarity setting min_periods to 100.

This process will also take a few minutes. Print the first 5 rows of the resulting matrix when done.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to compute all correlations between columns (movies) in the matrix user_movie, but considering only movies having at least 100 ratings in common. Store this in "item_similarity_min_ratings"</font>

In [55]:
item_similarity_min_ratings = user_movie.corr(min_periods=100)

In [56]:
display(item_similarity_min_ratings.head(5))

movie_id,2769,3177,3190,3225,3228,3239,3273,3275,3276,3279,...,33138,33145,33148,33150,33152,33154,33158,33162,33164,33166
movie_id,,,,,,,,,,,,,,,,,,,,,
2769,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3177,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3190,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3225,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We will need to test our function so let us select a couple of interesting users.

Our first user, `user_id_super` will be someone who has given the following 3 films a rating higher than 4.5:

* movie_id=5349: *Spider-Man (2002)*
* movie_id=3793: *X-Men (2000)*
* movie_id=6534: *Hulk (2003)* 	

Our second user, `user_id_drama` will be someone who has given the following 3 films a rating higher than 4.5:

* movie_id=6870: *Mystic River (2003)*
* movie_id=5995: *Pianist, The (2002)*
* movie_id=3555: *U-571 (2000)*

To filter a dataframe by multiple conditions you can use, e.g., `df[(a > 1) & (b > 2)]`. 

**Important**: these particular users have watched lots of movies, so we cannot tell for sure they have only these interests.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to find the userids of two example users: user_id_super (the who liked the three superhero movies), and user_id_drama (the one who liked the three dramas)</font>

In [ ]:
user_id_super = ratings.loc[ratings.loc[(ratings['movie_id'] == 5349)].rating>4.5 & ratings.loc[(ratings['movie_id'] == 3793)].rating>4.5]
user_id_drama = ratings


We will need some auxiliary functions that are provided below. You can leave as-is.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [24]:
# Leave this code as-is

# Gets a list of watched movies for a user_id
def get_watched_movies(user_id, user_movie):
    return list(user_movie.loc[user_id].dropna().sort_values(ascending=False).index)
    
# Gets the rating a user_id has given to a movie_id
def get_rating(user_id, movie_id, user_movie):
    return user_movie[movie_id][user_id]

# Print watched movies
def print_watched_movies(user_id, user_movie, movies):
    for movie_id in get_watched_movies(user_id, user_movie):
        print("%d %.1f %s " %
          (movie_id, get_rating(user_id, movie_id, user_movie), get_title(movie_id, movies)))


In [25]:
# LEAVE AS-IS (TESTING CODE)

print_watched_movies(user_id_super, user_movie, movies)

5502 5.0 Signs (2002) 
5445 5.0 Minority Report (2002) 
6156 5.0 Shanghai Knights (2003) 
5952 5.0 Lord of the Rings: The Two Towers, The (2002) 
5944 5.0 Star Trek: Nemesis (2002) 
5816 5.0 Harry Potter and the Chamber of Secrets (2002) 
5618 5.0 Spirited Away (Sen to Chihiro no kamikakushi) (2001) 
5524 5.0 Blue Crush (2002) 
5480 5.0 Stuart Little 2 (2002) 
5459 5.0 Men in Black II (a.k.a. MIIB) (a.k.a. MIB 2) (2002) 
5420 5.0 Windtalkers (2002) 
4388 5.0 Scary Movie 2 (2001) 
5389 5.0 Spirit: Stallion of the Cimarron (2002) 
5349 5.0 Spider-Man (2002) 
5218 5.0 Ice Age (2002) 
5064 5.0 The Count of Monte Cristo (2002) 
4993 5.0 Lord of the Rings: The Fellowship of the Ring, The (2001) 
4973 5.0 Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le) (2001) 
4896 5.0 Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001) 
4886 5.0 Monsters, Inc. (2001) 
6186 5.0 Gods and Generals (2003) 
6333 5.0 X2: X-Men United (2003) 
6377 5.0 Finding Nemo (2003) 
6

In [26]:
# LEAVE AS-IS (TESTING CODE)

print_watched_movies(user_id_drama, user_movie, movies)

3967 5.0 Billy Elliot (2000) 
4014 5.0 Chocolat (2000) 
4034 5.0 Traffic (2000) 
5995 5.0 Pianist, The (2002) 
7147 5.0 Big Fish (2003) 
4995 5.0 Beautiful Mind, A (2001) 
3555 5.0 U-571 (2000) 
6870 5.0 Mystic River (2003) 
5991 5.0 Chicago (2002) 
8464 5.0 Super Size Me (2004) 
5669 5.0 Bowling for Columbine (2002) 
8622 5.0 Fahrenheit 9/11 (2004) 
30707 5.0 Million Dollar Baby (2004) 
6953 4.5 21 Grams (2003) 
5015 4.5 Monster's Ball (2001) 
5464 4.5 Road to Perdition (2002) 
3510 4.5 Frequency (2000) 
5989 4.5 Catch Me If You Can (2002) 
4022 4.0 Cast Away (2000) 
5010 4.0 Black Hawk Down (2001) 
5299 4.0 My Big Fat Greek Wedding (2002) 
3897 4.0 Almost Famous (2000) 
3755 4.0 Perfect Storm, The (2000) 
4308 4.0 Moulin Rouge (2001) 
4447 3.5 Legally Blonde (2001) 
4246 3.5 Bridget Jones's Diary (2001) 
4975 3.5 Vanilla Sky (2001) 
4019 3.5 Finding Forrester (2000) 
5377 3.5 About a Boy (2002) 
3948 3.5 Meet the Parents (2000) 
5956 3.0 Gangs of New York (2002) 
6281 3.0 Phone Booth

For every user, we will consider that the importance of a new movie (a movie s/he has not rated) will be equal to the sum of the similarities between that new movie and all the movies the user has already rated.

Indeed, to further improve this, we will compute a weighted sum, in which the weight will be the rating given to the movie.

For instance, suppose a user has rated movies as follows:

```
movie_id rating
1        2.0
2        3.0
3        NaN
4        NaN
```

And that movie similarities are as follows (values with a "." do not matter in this example):

```
movie_id   1   2   3   4
1         ...............
2         ...............
3         0.1 0.2 NaN ...
4         0.9 0.8 ... NaN
```

The importance of movie 3 to this user will be:

```
2.0 * 0.1 + 3.0 * 0.2 = 0.8
```

While the importance of movie 4 to this user will be:

```
2.0 * 0.9 + 3.0 + 0.8 = 5.6
```

As we can see, we are favoring movies that are highly similar to many movies that the user has rated high.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

Create a function `get_movies_relevance` that returns a dataframe with columns `movie_id` and `relevance`. You can use the following template:

```python
def get_movies_relevance(user_id, user_movie, item_similarity_matrix):
    
    # Create an empty series
    movies_relevance = ...
    
    # Iterate through the movies the user has watched
    for watched_movie in ...
        
        # Obtain the rating given
        rating_given = ...
        
        # Obtain the vector containing the similarities of watched_movie
        # with all other movies in item_similarity_matrix
        similarities = ...
        
        # Multiply this vector by the given rating
        weighted_similarities = ...
        
        # Append these terms to movies_relevance
        movies_relevance = movies_relevance.append(weighted_similarities)
    
    # Compute the sum for each movie
    movies_relevance = movies_relevance.groupby(movies_relevance.index).sum()
    
    # Convert to a dataframe
    movies_relevance_df = pd.DataFrame(movies_relevance, columns=['relevance'])
    movies_relevance_df['movie_id'] = movies_relevance_df.index
    
    return movies_relevance_df

```

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code for "get_movies_relevance"</font>

In [ ]:
def get_movies_relevance(user_id, user_movie, item_similarity_matrix):

    # Create an empty series
    movies_relevance = ...

    # Iterate through the movies the user has watched
    for watched_movie in ...

        # Obtain the rating given
        rating_given = ...

        # Obtain the vector containing the similarities of watched_movie
        # with all other movies in item_similarity_matrix
        similarities = ...

        # Multiply this vector by the given rating
        weighted_similarities = ...

        # Append these terms to movies_relevance
        movies_relevance = movies_relevance.append(weighted_similarities)

    # Compute the sum for each movie
    movies_relevance = movies_relevance.groupby(movies_relevance.index).sum()

    # Convert to a dataframe
    movies_relevance_df = pd.DataFrame(movies_relevance, columns=['relevance'])
    movies_relevance_df['movie_id'] = movies_relevance_df.index

    return movies_relevance_df


Apply `get_movies_relevance` to the two users we have selected, `user_id_super` and `user_id_drama`.

The result will contain only `movie_id` and `relevance`, you will have to merge with the `movies` dataframe on the `movie_id` attribute.

Sort the results by descending relevance and print the top 10 for each case.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to obtain the 10 most relevant movies for the users user_id_super (who likes superhero movies) and user_id_drama (who likes dramas)</font>

<font size="+1" color="red">Replace this cell with a brief commentary on the movies you see on these lists. How many of them look relevant for the intended users? Feel free to use IMDB or Wikipedia to get info on these movies.</font>

<font size="-1" color="gray">All those trivial facts you learned about 1980s and 1990s pop culture were supposed to be useful one day; that day has arrived :-)</font>

Finally, you only need to remove the movies the user has watched. To do so:

* Obtain the dataframe of relevant movies with `get_movies_relevance`
* Set this dataframe index to 'movie_id'
* Obtain the list of movie_ids of watched movies with `get_watched_movies`
* Drop from the relevant movies dataframe the watched movies

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code implementing "get_recommended_movies"</font>

<font size="+1" color="red">Replace this cell with your code to obtain the 10 most recommended movies for the users user_id_super and user_id_drama</font>

<font size="+1" color="red">Replace this cell with a brief commentary on these recommendations. Do you think they are relevant? Why or why not? After removing the movies the user has already watched, are the relevance scores of the remaining items comparable to the previous lists that contained all relevant movies?</font>

# DELIVER (individually)

Remember to read the section on "delivering your code" in the [course evaluation guidelines](https://github.com/chatox/data-mining-course/blob/master/upf/upf-evaluation.md).

Deliver a zip file containing:

* This notebook

## Extra points available

For more learning and extra points, use the [surprise](http://surpriselib.com/) library to generate recommendations for the same two users. Display the generated recommendations and comment on them.

**Note:** if you go for the extra points, add ``<font size="+2" color="blue">Additional results: surprise library</font>`` at the top of your notebook.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>